# Setup

In [ ]:
!pip install scikeras

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from scikeras.wrappers import KerasClassifier
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import shutil

from google.colab import drive

In [ ]:
import torch
import numpy as np

# Dataset

In [ ]:
#Replace to the working datapath
drive.mount('/content/drive')
train_path = ''
test_path = ''
val_path = ''

## Preprocess the dataset

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=(256, 256),
    batch_size=32,
    shuffle=True
)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=(256, 256)
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_path,
    image_size=(256, 256)
)

In [ ]:
def map(image, label):
  num_classes = 6
  label = to_categorical(label, num_classes=num_classes)
  return image, label

In [ ]:
train_ds = train_ds.map(map)
val_ds   = val_ds.map(map)
test_ds  = test_ds.map(map)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break
print(labels_batch)

In [ ]:
def augment(image, label):
    scale_factor = tf.random.uniform([], 0.5, 2.0)
    orig_shape = tf.shape(image)[1:3]
    scaled_shape = tf.cast(tf.cast(orig_shape, tf.float32) * scale_factor, tf.int32)
    resized_image = tf.image.resize(image, scaled_shape)
    rescaled_image = tf.image.resize(resized_image, orig_shape)

    return rescaled_image, label

In [ ]:
train_ds = train_ds.map(augment)
val_ds = val_ds.map(augment)
test_ds  = test_ds.map(augment)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  print(argmax(labels_batch))
  break
print(labels_batch)

# Model

## Prepare

In [ ]:
output_size = 6
batch_size = 32
max_epoches = 100

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  patience=3,
                                                  restore_best_weights=True,
                                                  verbose=1)

In [ ]:
# Customize based on need
METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.CategoricalCrossentropy(name='cross entropy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

## Model initialized

In [ ]:
model_1 = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),

  tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Conv2D(32, 3, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
  tf.keras.layers.MaxPooling2D(),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)),
  tf.keras.layers.Dense(6, activation="softmax")
])

In [ ]:
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss='categorical_crossentropy',
                metrics=METRICS)

## Start

In [ ]:
history_1 = model_1.fit(train_ds,
                      batch_size=batch_size,
                      callbacks=[early_stopping],
                      epochs = max_epoches,
                      validation_data=(val_ds),
                      verbose=2)

## Evaluate

In [ ]:
model_1.evaluate(test_ds, verbose=2)

In [ ]:
plt.plot(history_1.history['accuracy'], label='Training Accuracy')
plt.plot(history_1.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history_1.history['loss'], label='Training Loss')
plt.plot(history_1.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

# Plot diagram

In [ ]:
import seaborn as sns

In [ ]:
model = tf.keras.models.load_model('') # If want to test saved models

In [ ]:
class_names = test_ds.class_names
class_count = len(class_names)

In [ ]:
label_map = {class_name: i for i, class_name in enumerate(class_names)}

In [ ]:
y_true = []
y_pred = []

for images, labels in test_ds:
    predictions = model.predict(images)
    predicted_labels = np.argmax(predictions, axis=1)

    true_labels = [label_map[class_names[label]] for label in labels.numpy()]

    y_true.extend(true_labels)
    y_pred.extend(predicted_labels)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()